In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import ast
from torch.utils.data import DataLoader
import re

from coral import interactive
from coral.dataset.parse_python import parse_string, extract_funcs_from_py
from coral.dataset import DataReader, UnitedVocab, CORALDataset, my_collate, key_lib

In [3]:
session = interactive.Session(dataset="/projects/bdata/jupyter/gezhang_backup/jupyter-notebook-analysis/graphs/cell_with_func_python23_1_27.txt",
                              model_path='/projects/bdata/jupyter/gezhang_backup/BERT-pytorch/pretrain/model.ep9',
                              vocab_path='/projects/bdata/jupyter/gezhang_backup/BERT-pytorch/pretrain/vocab.txt',
                              with_cuda = False)

Load Data /projects/bdata/jupyter/gezhang_backup/jupyter-notebook-analysis/graphs/cell_with_func_python23_1_27.txt


2516572it [02:29, 16788.97it/s]


Loading Vocab
Vocab Size:  17654
Loading Train Dataset /projects/bdata/jupyter/gezhang_backup/jupyter-notebook-analysis/graphs/cell_with_func_python23_1_27.txt
698074
Loading Dev Dataset /projects/bdata/jupyter/gezhang_backup/jupyter-notebook-analysis/graphs/cell_with_func_python23_1_27.txt
174519
Creating Dataloaders
Building BERT model
Creating BERT Trainer
Total Parameters: 12742902
Trainer Complete


In [27]:
def get_string_between(s,start,stop):
    result = re.search('{0}(.*){1}'.format(start,stop), s)
    return(result.group(1))

def get_string_before(s,stop):
    index = s.find(stop)
    return s[:index]

def get_string_after(s,start):
    len_start = len(start)
    index = s.rfind(start)
    return s[(index + len_start):]


def snippet_to_graph(snippet,mask_start = None, mask_end=None):
    
    if mask_start and mask_end:
        start = get_string_before(snippet,mask_start)
        to_mask = get_string_between(snippet,mask_start,mask_end)

        n_nodes_in_start = len(parse_string(start))
        n_nodes_in_mask = len(parse_string(to_mask)) - 1
        
        snippet = snippet.replace(mask_end,"").replace(mask_start,"")
        print(snippet)
        
        to_mask = list(range(n_nodes_in_start, n_nodes_in_start + n_nodes_in_mask))
    
    else:
        to_mask = None
        
        
    graph = {
        "context": snippet,
        "nodes"  : parse_string(snippet),
        "funcs"  : extract_funcs_from_py(snippet)[0],
        "annotation" : [],
        "header": "[EMPTY]",
        "to_mask": to_mask
    }
    
    return graph


def decode_sequence(seq):
    return [session.vocab.idx2word[x] for x in seq]

def snippet_forward(session,snippet, masked = True,mask_start = None, mask_end=None,
                   mask_fn = None):
    
    graph = snippet_to_graph(snippet,mask_start = mask_start, mask_end=mask_end)
    
    
    #Do we need to do this step?
    #Yes, because this is where we get the adj matrix
    dataset = CORALDataset([graph], session.vocab, seq_len=session.seq_len,
              n_neg=session.duplicate, use_sub_token=session.use_sub_token, 
              markdown=session.markdown, masked=True, mask_fn = mask_fn)
    data_loader = DataLoader(dataset,collate_fn=my_collate)
    results, topk  = session.trainer.api(data_loader)
    
    
    
    mask_results = []
    ii = 0
    # for (d, _), r, tk in zip(test_dataset, results, topk):
    for (d, _), r, tk in zip(dataset, results, topk):

        # g = test_dataset.graphs[ii]
        g = dataset.graphs[ii]

        ii += 1
        bert_input = d["bert_input"]
        target_label = d["target_label"]
        flag = False
        for i, t in enumerate(bert_input):

            if t == 2:
                if not flag:
                    print('=' * 20)
                    print(g["context"])
                    flag = True
                mask_results.append((target_label[i], r[i]))

                print(session.vocab.idx2word[target_label[i]], [
                      session.vocab.idx2word[tkk] for tkk in tk[i]])

    counter = 0
    for t, r in mask_results:
        if t == r:
            counter += 1

    tokens = []
    for t, r in mask_results:

        if session.vocab.idx2word[r] not in tokens:
            tokens.append(session.vocab.idx2word[r])
def decode_sequence(seq):
    return [session.vocab.idx2word[x] for x in seq]

In [1]:
custom_snippet = """from sklearn import linear_model
reg = sklearn.linear_model.LinearRegression()
reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])

reg.coef_"""
snippet_forward(session,custom_snippet)

NameError: name 'snippet_forward' is not defined

In [21]:
def roll_your_own_mask(graph):
    nodes = session.test_dataset.get_node_seq(graph)
    to_mask = graph["to_mask"]
    masked_nodes = []
    for i,n in enumerate(nodes):
        if i in to_mask:
            masked_nodes.append("[MASK]")
        else:
            masked_nodes.append(n)
    ids = [session.vocab.word2idx.get(n, session.vocab.unk_index) for n in masked_nodes]
    return ids

In [26]:
masked_snippet = """from sklearn import linear_model
reg = sklearn.linear_model.LinearRegression()
<mask>reg.fit(X)</mask>
reg.coef_
"""

masked_graph = snippet_to_graph(masked_snippet,mask_start="<mask>", mask_end = "</mask>")
print(roll_your_own_mask(masked_graph))

from sklearn import linear_model
reg = sklearn.linear_model.LinearRegression()
reg.fit(X)
reg.coef_

[10511, 4855, 15467, 3243, 1186, 3160, 3745, 3745, 4855, 15467, 12876, 2, 2, 2, 2, 2, 2, 13842, 3745, 1186, 588]


In [31]:
masked_snippet = """from sklearn import linear_model

reg = sklearn.linear_model.LinearRegression()
reg.fit(X)
<mask>print(X)</mask>
reg.coef_
"""
snippet_forward(session,masked_snippet,mask_start = "<mask>", mask_end = "</mask>", mask_fn=roll_your_own_mask)

100%|| 1/1 [00:00<00:00, 11.15it/s]

from sklearn import linear_model

reg = sklearn.linear_model.LinearRegression()
reg.fit(X)
print(X)
reg.coef_

from sklearn import linear_model

reg = sklearn.linear_model.LinearRegression()
reg.fit(X)
print(X)
reg.coef_

Expr ['LinearRegression', 'Ridge', 'mean_squared_error', 'Lasso', 'r2_score']
Call ['LinearRegression', 'mean_squared_error', 'Ridge', 'Lasso', 'r2_score']
print ['LinearRegression', 'Ridge', 'mean_squared_error', 'Lasso', 'r2_score']
X ['LinearRegression', 'Ridge', 'mean_squared_error', 'Lasso', 'r2_score']


In [33]:
masked_snippet_prime = """
le = preprocessing.LabelEncoder()
<mask>titanic_data['Sex'] = le.fit_transform(titanic_data['Sex'])</mask>
titanic_data['Embarked'] = le.fit_transform(titanic_data['Embarked'])
titanic_data['name_title'] = le.fit_transform(titanic_data['name_title'])
titanic_data.head()
"""
snippet_forward(session,masked_snippet_prime,mask_start = "<mask>", mask_end = "</mask>", mask_fn=roll_your_own_mask)

100%|| 1/1 [00:00<00:00,  8.05it/s]


le = preprocessing.LabelEncoder()
titanic_data['Sex'] = le.fit_transform(titanic_data['Sex'])
titanic_data['Embarked'] = le.fit_transform(titanic_data['Embarked'])
titanic_data['name_title'] = le.fit_transform(titanic_data['name_title'])
titanic_data.head()


le = preprocessing.LabelEncoder()
titanic_data['Sex'] = le.fit_transform(titanic_data['Sex'])
titanic_data['Embarked'] = le.fit_transform(titanic_data['Embarked'])
titanic_data['name_title'] = le.fit_transform(titanic_data['name_title'])
titanic_data.head()

Assign ['LabelEncoder', 'apply', 'inverse_transform', 'Series', 'crosstab']
SubscriptStore ['LabelEncoder', 'apply', 'sum', 'inverse_transform', 'skew']
titanic_data ['LabelEncoder', 'apply', 'inverse_transform', 'PCA', 'map']
Index ['LabelEncoder', 'apply', 'map', 'inverse_transform', 'sum']
Sex ['LabelEncoder', 'apply', 'inverse_transform', 'PCA', 'map']
Call ['apply', 'map', 'LabelEncoder', 'inverse_transform', 'face']
AttributeLoad ['LabelEncoder', 'apply', 'map', 'skew',

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
"ab<mask>c</mask>".find("<mask>")

In [ ]:
"ab<mask>c</mask>"[:2]

In [ ]:
list(range(10,12))

In [ ]:
n_mask = 2
n_before =  2
"a b d c e".split()[n_before:n_before+ n_mask]

In [ ]:
list(range(n_before, n_before + n_mask))

In [ ]:
"a b d c e".split()[3]